# Buchi automata complementation

In [ ]:
# PREAMBLE
import altair as alt
import pandas as pd
import re as re

FILE_INPUT = "results/random-to300.csv"


# do not care about the limit
alt.data_transformers.disable_max_rows()
#alt.renderers.enable('mimetype')
#alt.renderers.enable('altair_viewer')
#alt.renderers.enable('html')
#alt.renderers.enable('notebook')
#alt.renderers.enable('mimebundle')




# For reading in files
def read_file(filename):
   """Reads a CSV file into Panda's data frame"""
   df = pd.read_csv(
          filename,
          sep=";",
          comment="#",
          na_values=['ERR','TO'])
   return df 


# For printing scatter plots
def scatter_plot(df, xcol, ycol, domain, log=False):
    assert len(domain) == 2
    plot_type = "log" if log else "linear"
    scatter = alt.Chart(df).mark_point(size=10, filled=True).encode(
       x=alt.X(xcol + ':Q',scale=alt.Scale(type=plot_type, base=10, domain=domain)),
       y=alt.Y(ycol + ':Q',scale=alt.Scale(type=plot_type, base=10, domain=domain))
       )

    rules = (alt.Chart(pd.DataFrame({'y': [domain[1]]})).mark_rule(strokeDash=[3,1]).encode(y='y') +
             alt.Chart(pd.DataFrame({'x': [domain[1]]})).mark_rule(strokeDash=[3,1]).encode(x='x'))

    diag = alt.Chart(pd.DataFrame({'x': domain, 'y': domain})).mark_line(color='black', strokeDash=[3,1], size=1).encode(x='x', y='y')

    res = scatter + rules + diag
    res = res.properties(
       width=600, height=600
    )

    return res


# Print a matrix of plots
def matrix_plot(list_of_plots, cols):
  assert len(list_of_plots) > 0
  assert cols >= 0

  matrix_plot = None
  row = None
  for i in range(0, len(list_of_plots)):
    if i % cols == 0:   # starting a new row
      row = list_of_plots[i]
    else:
      row |= list_of_plots[i]

    if (i+1) % cols == 0 or i + 1 == len(list_of_plots): # last chart in a row
      if not matrix_plot:   # first row finished
        matrix_plot = row
      else:
        matrix_plot &= row

  return matrix_plot

In [ ]:

df = read_file(FILE_INPUT)

summary_states = dict()
for col in df.columns:
  if re.search('-States$', col):
    summary_states[col] = dict()
    summary_states[col]['max'] = df[col].max()
    summary_states[col]['min'] = df[col].min()

print(summary_states)

df_summary_states = pd.DataFrame(summary_states).transpose()
print(df_summary_states)

states_min = 1
states_max = df_summary_states['max'].max()
states_timeout = states_max * 1.1

# sanitizing NAs
for col in df.columns:
  if re.search('-States$', col):
    df[col].fillna(states_timeout, inplace=True)
    df[col].replace(0,states_min, inplace=True)   # to remove 0 (in case of log graph)

print("States min: {}".format(states_min))
print("States max: {}".format(states_max))

to_cmp = [
  ('ranker', 'seminat'),
  ('ranker', 'safra'),
  ('ranker', 'goal-df'),
  ('ranker', 'piterman'),
  ('ranker', 'schewe'),
  ('ranker', 'spot'),
  ('ranker', 'roll'),
]

plot_list = [scatter_plot(df, xcol=x + '-States', ycol=y + '-States', domain=[states_min, states_timeout], log=True) for x, y in to_cmp]

mplot = matrix_plot(plot_list, cols=2)

mplot


In [ ]:
hard = read_file("ranker-hard-pure-sanitized.csv")
df = df.merge(hard)

df.loc[df['hard'], 'ranker+piterman'] = df['ranker-States']
df.loc[~df['hard'], 'ranker+piterman'] = df['piterman-States']

df.loc[df['hard'], 'ranker+spot'] = df['ranker-States']
df.loc[~df['hard'], 'ranker+spot'] = df['spot-States']

df.loc[df['hard'], 'ranker+safra'] = df['ranker-States']
df.loc[~df['hard'], 'ranker+safra'] = df['safra-States']

df.loc[df['hard'], 'ranker+seminat'] = df['ranker-States']
df.loc[~df['hard'], 'ranker+seminat'] = df['seminat-States']


comb_list = [scatter_plot(df, xcol='ranker+piterman', ycol='ranker-States', domain=[states_min, states_timeout], log=True),
             scatter_plot(df, xcol='ranker+spot', ycol='ranker-States', domain=[states_min, states_timeout], log=True),
             scatter_plot(df, xcol='ranker+safra', ycol='ranker-States', domain=[states_min, states_timeout], log=True),
             scatter_plot(df, xcol='ranker+seminat', ycol='ranker-States', domain=[states_min, states_timeout], log=True)
            ]

matrix_plot(comb_list, cols=2)





In [ ]:
to_cmp = [
  ('ranker+piterman', 'seminat-States'),
  ('ranker+piterman', 'safra-States'),
  ('ranker+piterman', 'goal-df-States'),
  ('ranker+piterman', 'piterman-States'),
  ('ranker+piterman', 'schewe-States'),
  ('ranker+piterman', 'spot-States'),
  ('ranker+piterman', 'roll-States'),
]

plot_list = [scatter_plot(df, xcol=x, ycol=y, domain=[states_min, states_timeout], log=True) for x, y in to_cmp]

mplot = matrix_plot(plot_list, cols=2)

mplot


In [ ]:
df[df['ranker+piterman'] == states_timeout].count()
